<a href="https://colab.research.google.com/github/hrbolek/functing/blob/main/notebooks/async_examples.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Examples of usage

At first little hack from https://stackoverflow.com/questions/64074295/is-there-an-equivalence-of-await-in-google-colab to allow async function evaluation in Google Colab.

In [3]:
!pip install nest_asyncio

In [5]:
import nest_asyncio
nest_asyncio.apply()

import asyncio
awaitFunc = lambda x: asyncio.get_event_loop().run_until_complete(x)

In [1]:
!pip install functing

  Created wheel for functing: filename=functing-0.4-cp37-none-any.whl size=2805 sha256=2a69de6b81e7ca18e4d36281fe8b2bf5f4fa9a92caf40241ce3a4b25b89b75f5
  Stored in directory: /root/.cache/pip/wheels/b2/d7/d1/597d072821914ef2a9ab2e9c94b272e29e5fa543640e641b2a
Successfully built functing


## Example A

In [7]:
from functing.asynchronous import createTag

Td = createTag('td')()

async def Card(name):
    return Td(
        Td(
            'Hello', name
        )
    )

pageCode = awaitFunc(Card('John')) # usually pageCode = await Card('John')
html = awaitFunc(pageCode())       # usually html = await pageCode()
print(html)

['<td>', '<td>', 'Hello', 'John', '</td>', '</td>']


## Example B

In [8]:
import time

def mS(start=0):
    return time.time() - start

In [16]:
from functing.asynchronous import Children

Td = createTag('td')(className='card')
Tr = createTag('tr')()
Th = createTag('th')()
Table = createTag('table')()
TableBody = createTag('tbody')()

async def TableCell(data):
    await asyncio.sleep(1)
    return Td(data)

def Head(data):
    result = [Th(col) for col in data.keys()]
    return Tr(*result)

def Row(data):
    result = [TableCell(str(value)) for value in data.values()]
    return Tr(*result)

def FullTable(dataRows):
    bodyRows = [Row(data) for data in dataRows]
    return Table(
        Head(dataRows[0]),
        TableBody(*bodyRows)
    )

Notice that `TableCell` is called 12 times. With blocking call it should take more than 12 second. As asynchronous templating generates `TableCell` concurently, it takes only approximately 1 second.

In [17]:
subjectTable = [
    {'id': 1024, 'name': 'Mathematics', 'lessons': 60},
    {'id': 1144, 'name': 'English', 'lessons': 30},
    {'id': 1194, 'name': 'History', 'lessons': 75},
    {'id': 1086, 'name': 'Physics', 'lessons': 45},
]

start = mS()
pageCode = FullTable(subjectTable) 
htmlItems = awaitFunc(pageCode())  # htmlItems = await pageCode()
htmlStr = ''.join(htmlItems)
duration = mS(start)
print('duration', duration, 'ms')
print(htmlItems)
print(htmlStr.replace('<tr>', '\n<tr>'))


duration 1.0039935111999512 ms
['<table>', '<tr>', '<th>', 'id', '</th>', '<th>', 'name', '</th>', '<th>', 'lessons', '</th>', '</tr>', '<tbody>', '<tr>', '<td className="card">', '1024', '</td>', '<td className="card">', 'Mathematics', '</td>', '<td className="card">', '60', '</td>', '</tr>', '<tr>', '<td className="card">', '1144', '</td>', '<td className="card">', 'English', '</td>', '<td className="card">', '30', '</td>', '</tr>', '<tr>', '<td className="card">', '1194', '</td>', '<td className="card">', 'History', '</td>', '<td className="card">', '75', '</td>', '</tr>', '<tr>', '<td className="card">', '1086', '</td>', '<td className="card">', 'Physics', '</td>', '<td className="card">', '45', '</td>', '</tr>', '</tbody>', '</table>']
<table>
<tr><th>id</th><th>name</th><th>lessons</th></tr><tbody>
<tr><td className="card">1024</td><td className="card">Mathematics</td><td className="card">60</td></tr>
<tr><td className="card">1144</td><td className="card">English</td><td classNam

## Example C

Now the main template is async function.

In [18]:
from functing.asynchronous import Children

Td = createTag('td')(className='card')
Tr = createTag('tr')()
Th = createTag('th')()
Table = createTag('table')()
TableBody = createTag('tbody')()

async def TableCell(data):
    await asyncio.sleep(1)
    return Td(data)

def Head(data):
    result = [Th(col) for col in data.keys()]
    return Tr(*result)

def Row(data):
    result = [TableCell(str(value)) for value in data.values()]
    return Tr(*result)

async def FullTable(dataRows):
    bodyRows = [Row(data) for data in dataRows]
    return Table(
        Head(dataRows[0]),
        TableBody(*bodyRows)
    )

In [19]:
subjectTable = [
    {'id': 1024, 'name': 'Mathematics', 'lessons': 60},
    {'id': 1144, 'name': 'English', 'lessons': 30},
    {'id': 1194, 'name': 'History', 'lessons': 75},
    {'id': 1086, 'name': 'Physics', 'lessons': 45},
]

start = mS()
pageCode = awaitFunc(FullTable(subjectTable)) # pageCode = await FullTable(subjectTable)
htmlItems = awaitFunc(pageCode())             # htmlItems = await pageCode()
htmlStr = ''.join(htmlItems)
duration = mS(start)
print('duration', duration, 'ms')
print(htmlItems)
print(htmlStr.replace('<tr>', '\n<tr>'))


duration 1.0033257007598877 ms
['<table>', '<tr>', '<th>', 'id', '</th>', '<th>', 'name', '</th>', '<th>', 'lessons', '</th>', '</tr>', '<tbody>', '<tr>', '<td className="card">', '1024', '</td>', '<td className="card">', 'Mathematics', '</td>', '<td className="card">', '60', '</td>', '</tr>', '<tr>', '<td className="card">', '1144', '</td>', '<td className="card">', 'English', '</td>', '<td className="card">', '30', '</td>', '</tr>', '<tr>', '<td className="card">', '1194', '</td>', '<td className="card">', 'History', '</td>', '<td className="card">', '75', '</td>', '</tr>', '<tr>', '<td className="card">', '1086', '</td>', '<td className="card">', 'Physics', '</td>', '<td className="card">', '45', '</td>', '</tr>', '</tbody>', '</table>']
<table>
<tr><th>id</th><th>name</th><th>lessons</th></tr><tbody>
<tr><td className="card">1024</td><td className="card">Mathematics</td><td className="card">60</td></tr>
<tr><td className="card">1144</td><td className="card">English</td><td classNam